<a href="https://colab.research.google.com/github/ManinderSingh4000/AI_ML/blob/main/Guit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
import numpy as np

# Parameters
AUDIO_SEQ_LEN = 2000   # from your MFCC/audio features
AUDIO_FEATURE_DIM = 64
MAX_TARGET_LEN = 100    # token sequence length of XML/tab
VOCAB_SIZE = 500        # depends on tokenizer used on target text
EMBED_DIM = 256         # transformer model dimensionality
NUM_HEADS = 4
FF_DIM = 512
NUM_LAYERS = 4
DROPOUT_RATE = 0.1

In [2]:
class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, x, training):
        attn_output = self.att(x, x)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TransformerDecoderBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att1 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.att2 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.dropout3 = Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask=None, padding_mask=None):
        attn1 = self.att1(x, x, attention_mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(x + attn1)

        attn2 = self.att2(out1, enc_output)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)


In [3]:
import tensorflow as tf
import numpy as np

def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    # Apply sin to even indices, cos to odd indices
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, max_len, d_model):
        super().__init__()
        self.token_embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.pos_encoding = get_positional_encoding(max_len, d_model)

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.token_embedding(x)
        return x + self.pos_encoding[:, :length, :]


In [4]:
def build_transformer_model(
    audio_seq_len=2000,
    audio_feat_dim=64,
    max_target_len=100,
    vocab_size=500,
    embed_dim=256,
    ff_dim=512,
    num_heads=4,
    num_layers=4,
    dropout_rate=0.1,
):
    # ==== AUDIO ENCODER ====
    encoder_input = Input(shape=(audio_seq_len, audio_feat_dim), name="audio_input")
    x = Dense(embed_dim)(encoder_input)  # Project audio features to model dimension
    x += get_positional_encoding(audio_seq_len, embed_dim)[:, :audio_seq_len, :]

    for _ in range(num_layers):
        x = TransformerEncoderBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x, training=True)
    encoder_output = x

    # ==== TEXT DECODER ====
    decoder_input = Input(shape=(max_target_len,), name="decoder_input")  # token ids
    y = PositionalEmbedding(vocab_size, max_target_len, embed_dim)(decoder_input)

    for _ in range(num_layers):
        y = TransformerDecoderBlock(embed_dim, num_heads, ff_dim, dropout_rate)(
            y, encoder_output, training=True
        )

    # Output logits over vocabulary
    decoder_output = Dense(vocab_size, activation='softmax')(y)

    model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_output)
    return model


In [5]:
transformer = build_transformer_model()

transformer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [6]:
transformer.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ audio_input         │ (None, 2000, 64)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 2000, 256) │     16,640 │ audio_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 2000, 256) │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 2000, 256) │  1,315,840 │ add[0][0]         │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 2000, 256) │  1,315,840 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 2000, 256) │  1,315,840 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, 100, 256)  │    128,000 │ decoder_input[0]… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 2000, 256) │  1,315,840 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, 100, 256)  │  2,368,256 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, 100, 256)  │  2,368,256 │ transformer_deco… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, 100, 256)  │  2,368,256 │ transformer_deco… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, 100, 256)  │  2,368,256 │ transformer_deco… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 100, 500)  │    128,500 │ transformer_deco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,009,524 (57.26 MB)

 Trainable params: 15,009,524 (57.26 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
def prepare_decoder_inputs_outputs(tokenized_targets, start_token_id):
    """
    Takes target sequences (e.g., [34, 76, 12, 9, ...]) and returns:
    decoder_input: [<start>, 34, 76, 12, ...]
    decoder_output: [34, 76, 12, 9, ...]
    """
    decoder_inputs = []
    decoder_outputs = []

    for seq in tokenized_targets:
        decoder_inputs.append([start_token_id] + seq[:-1])
        decoder_outputs.append(seq)

    return np.array(decoder_inputs), np.array(decoder_outputs)


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Assume you have a list of string targets (e.g., XML or tablature format)
# Example:
text_targets = [
    "<note><onsetSec>6.605</onsetSec></note>",
    "<note><onsetSec>7.230</onsetSec></note>",
    "E|--0--2--3--|",
    # etc.
]

# Use character-level tokenizer (or switch to word-level if preferred)
tokenizer = Tokenizer(char_level=True, filters='', lower=False)
tokenizer.fit_on_texts(text_targets)

# Convert each string to a list of token IDs
Y_token = tokenizer.texts_to_sequences(text_targets)

# Set a maximum length for output sequences
MAX_TARGET_LEN = 100
Y_token = pad_sequences(Y_token, maxlen=MAX_TARGET_LEN, padding='post')

# Update vocabulary size
VOCAB_SIZE = len(tokenizer.word_index) + 1
start_token_id = tokenizer.word_index.get('<', 1)  # use `<` as start token


In [9]:
def prepare_decoder_inputs_outputs(tokenized_targets, start_token_id):
    decoder_inputs = []
    decoder_outputs = []

    for seq in tokenized_targets:
        decoder_input = [start_token_id] + list(seq[:-1])  # shift right
        decoder_output = list(seq)
        decoder_inputs.append(decoder_input)
        decoder_outputs.append(decoder_output)

    return np.array(decoder_inputs), np.array(decoder_outputs)

# Apply function
decoder_input_data, decoder_output_data = prepare_decoder_inputs_outputs(Y_token, start_token_id)

# decoder_output needs to be 3D for sparse_categorical_crossentropy
decoder_output_data = decoder_output_data[..., np.newaxis]


In [10]:
import os
import librosa
import numpy as np

# Directory containing your .wav files
audio_dir = "/content/drive/MyDrive/dataset2/audio"  # change as needed
files = [f for f in os.listdir(audio_dir) if f.endswith(".wav")]

# Parameters
MAX_AUDIO_LEN = 2000  # pad/truncate to 2000 time steps
N_MFCC = 64           # number of MFCCs to extract

def extract_mfcc(file_path, max_len=MAX_AUDIO_LEN, n_mfcc=N_MFCC):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc = mfcc.T  # shape: (time, 64)

    if mfcc.shape[0] < max_len:
        # Pad with zeros if shorter
        pad_width = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
    else:
        # Truncate if longer
        mfcc = mfcc[:max_len, :]

    return mfcc

# Load all MFCCs
X_audio = []
for fname in files:
    path = os.path.join(audio_dir, fname)
    mfcc = extract_mfcc(path)
    X_audio.append(mfcc)

X_audio = np.array(X_audio)  # final shape: (N, 2000, 64)


In [11]:
print(X_audio[0])

[[-5.3748553e+02  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-5.3748553e+02  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-5.3748553e+02  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 ...
 [-5.1853265e+02  2.5747753e+01  2.2964104e+01 ... -9.6628135e-01
  -1.3735790e+00 -1.3571057e+00]
 [-5.1610760e+02  2.8187939e+01  2.3252743e+01 ... -9.4099671e-01
  -1.5579896e+00 -1.6216792e+00]
 [-5.1643176e+02  2.7041553e+01  2.0601500e+01 ...  1.4030825e+00
   2.4551189e-01 -9.8784566e-02]]


In [12]:
import os
import librosa
import numpy as np
from bs4 import BeautifulSoup

# Paths to folders
audio_dir = "/content/drive/MyDrive/dataset2/audio"
xml_dir   = "/content/drive/MyDrive/dataset2/annotation"

X_audio = []
text_targets = []

# Step 1: Create a set of XML base names
xml_files = {os.path.splitext(f)[0] for f in os.listdir(xml_dir) if f.endswith(".xml")}

# Step 2: Iterate audio files and only include those with matching XML
for audio_file in os.listdir(audio_dir):
    if audio_file.endswith(".wav"):
        base_name = os.path.splitext(audio_file)[0]

        if base_name in xml_files:
            audio_path = os.path.join(audio_dir, audio_file)
            xml_path   = os.path.join(xml_dir, base_name + ".xml")

            # --- Process audio ---
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=64).T
            if mfcc.shape[0] < 2000:
                mfcc = np.pad(mfcc, ((0, 2000 - mfcc.shape[0]), (0, 0)), mode='constant')
            else:
                mfcc = mfcc[:2000, :]
            X_audio.append(mfcc)

            # --- Process XML ---
            with open(xml_path, "r", encoding="utf-8") as f:
                soup = BeautifulSoup(f, "xml")
                text_targets.append(str(soup))

# Convert to array
X_audio = np.array(X_audio)

# Final sanity check
print("✅ Matching files loaded:")
print(f" - Audio files: {len(X_audio)}")
print(f" - XML targets: {len(text_targets)}")


✅ Matching files loaded:
 - Audio files: 252
 - XML targets: 252


In [13]:
len(X_audio)

252

In [26]:
from transformers import AutoTokenizer

# Load pre-trained tokenizer (can use T5, BERT, GPT2, etc.)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Optional: Clean up any empty or malformed XML
text_targets = [txt.strip() for txt in text_targets if txt and len(txt.strip()) > 0]

# Critical: Match count of inputs and outputs
assert len(text_targets) == len(X_audio), f"Mismatch: {len(text_targets)} targets vs {len(X_audio)} audio"

# Tokenize all XMLs into input_ids
Y_token = tokenizer(
    text_targets,
    return_tensors="np",      # for NumPy (since X_audio is a NumPy array)
    padding="max_length",
    truncation=True,
    max_length=512            # or whatever your Transformer model expects
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# import os
# from bs4 import BeautifulSoup
# import xml.etree.ElementTree as ET

# # Directory with XML files
# xml_dir = "/content/drive/MyDrive/dataset2/annotation"

# # Get all XML file paths
# xml_files = [f for f in os.listdir(xml_dir) if f.endswith(".xml")]

# # Extract XML content as raw strings



# text_targets = []

# for xml_file in xml_files:
#     file_path = os.path.join(xml_dir, xml_file)

#     with open(file_path, "r", encoding="utf-8") as file:
#       xml_content = file.read()
#       print(xml_content)






        # soup = BeautifulSoup(file, "xml")

        # # Option 1: Keep full XML as text
        # text_targets.append(str(soup))

        # Option 2: If you only want specific tags like onsetSec, pitch, etc.:
        # notes = soup.find_all("note")
        # extracted = ""
        # for note in notes:
        #     extracted += str(note)  # or customize formatting
        # text_targets.append(extracted)
  # with open(file_path, 'r') as file:
          #  xml_content = file.read()

In [27]:
import os
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

# Directory with XML files
xml_dir = "/content/drive/MyDrive/dataset2/annotation"

# Get all XML file paths
xml_files = [f for f in os.listdir(xml_dir) if f.endswith(".xml")]

# Extract XML content as raw strings
text_targets = []

for xml_file in xml_files:
    file_path = os.path.join(xml_dir, xml_file)

    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "xml")

        # Option 1: Keep full XML as text
        text_targets.append(str(soup))

        # Option 2: If you only want specific tags like onsetSec, pitch, etc.:
        # notes = soup.find_all("note")
        # extracted = ""
        # for note in notes:
        #     extracted += str(note)  # or customize formatting
        # text_targets.append(extracted)
  # with open(file_path, 'r') as file:
          #  xml_content = file.read()

In [28]:
# List of XML strings, 1 per file
print(text_targets[20][:1500])  # show a snippet of the first


<?xml version="1.0" encoding="utf-8"?>
<instrumentRecording>
<globalParameter>
<audioFileName>\AR_Lick2_FN.wav</audioFileName>
<instrument>EGUI</instrument>
<instrumentModel>Aristides 010</instrumentModel>
<pickUpSetting>unknown</pickUpSetting>
<instrumentTuning>40  45  50  55  59  64</instrumentTuning>
<recordingDate>13-02-14</recordingDate>
<recordingArtist>Christian Kehling</recordingArtist>
<instrumentBodyMaterial>unknown</instrumentBodyMaterial>
<instrumentStringMaterial>Steel</instrumentStringMaterial>
<composer>Christian Kehling</composer>
</globalParameter>
<transcription>
<event>
<onsetSec>2.6712</onsetSec>
<pitch>56</pitch>
<offsetSec>2.9563</offsetSec>
<excitationStyle>FS</excitationStyle>
<expressionStyle>NO</expressionStyle>
<fretNumber>6</fretNumber>
<stringNumber>3</stringNumber>
<modulationFrequency>0</modulationFrequency>
<modulationFrequencyRange>0</modulationFrequencyRange>
</event>
<event>
<onsetSec>2.9723</onsetSec>
<pitch>57</pitch>
<offsetSec>3.2391</offsetSec>
<

In [29]:
import os
import librosa
import numpy as np
from bs4 import BeautifulSoup

# Set directories
audio_dir = "/content/drive/MyDrive/dataset2/audio"
xml_dir   = "/content/drive/MyDrive/dataset2/annotation"

# Storage lists
X_audio = []
text_targets = []

# Build a dict of XML files by basename
xml_map = {
    os.path.splitext(f)[0]: os.path.join(xml_dir, f)
    for f in os.listdir(xml_dir)
    if f.lower().endswith(".xml")
}

# Loop through audio files and find matching XML
for audio_file in os.listdir(audio_dir):
    if audio_file.lower().endswith(".wav"):
        base_name = os.path.splitext(audio_file)[0]

        # Check if XML with same basename exists
        if base_name in xml_map:
            audio_path = os.path.join(audio_dir, audio_file)
            xml_path   = xml_map[base_name]

            # -- Process Audio --
            y, sr = librosa.load(audio_path, sr=None)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=64).T
            if mfcc.shape[0] < 2000:
                mfcc = np.pad(mfcc, ((0, 2000 - mfcc.shape[0]), (0, 0)), mode='constant')
            else:
                mfcc = mfcc[:2000, :]
            X_audio.append(mfcc)

            # -- Process XML --
            with open(xml_path, "r", encoding="utf-8") as f:
                soup = BeautifulSoup(f, "xml")
                text_targets.append(str(soup))

# Convert to numpy
X_audio = np.array(X_audio)

# Check final stats
print("✅ Loaded:")
print(f"  - Audio files: {len(X_audio)}")
print(f"  - XML targets: {len(text_targets)}")
print(f"  - Shape of X_audio: {X_audio.shape}")


✅ Loaded:
  - Audio files: 252
  - XML targets: 252
  - Shape of X_audio: (252, 2000, 64)


In [30]:
count = 0
for f in os.listdir(audio_dir):
  count +=1
print(count)


261


In [37]:
count =0
for xml in os.listdir(xml_dir):
  count+=1

print(count)


261


##ERRor

In [38]:
# VOCAB_SIZE = len(tokenizer.word_index) + 1  # +1 for padding
VOCAB_SIZE = tokenizer.vocab_size  # This could be 30522 for BERT, etc.
VOCAB_SIZE


30522

In [39]:
# Step 1: Tokenize output (already discussed earlier)
Y_token = tokenizer(
    text_targets,
    return_tensors="np",
    padding="max_length",
    truncation=True,
    max_length=MAX_TARGET_LEN
)

# Step 2: Prepare decoder inputs and outputs
decoder_input_data = Y_token["input_ids"][:, :-1]       # shift left for input
decoder_output_data = Y_token["input_ids"][:, 1:]       # shift right for target

# Optional: pad so both have same max length
decoder_input_data = np.pad(decoder_input_data, ((0,0), (0,1)), constant_values=0)
decoder_output_data = np.pad(decoder_output_data, ((0,0), (0,1)), constant_values=0)


VOCAB_SIZE = tokenizer.vocab_size  # This could be 30522 for BERT, etc.


# Step 3: Build model (your function is good)
transformer = build_transformer_model(
    audio_seq_len=2000,
    audio_feat_dim=64,
    max_target_len=MAX_TARGET_LEN,
    vocab_size=VOCAB_SIZE,
    embed_dim= 128,
    ff_dim= 256 ,
    num_heads= 2 ,
    num_layers= 2 ,
    dropout_rate=0.1
)

# Step 4: Compile
transformer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Step 5: Train
transformer.fit(
    [X_audio, decoder_input_data],       # 2-input format
    decoder_output_data,
    batch_size= 4 ,
    epochs=20,
    validation_split=0.1
)


Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 342s 5s/step - accuracy: 0.0832 - loss: 9.9540 - val_accuracy: 0.1200 - val_loss: 8.8447
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 313s 5s/step - accuracy: 0.1200 - loss: 8.4791 - val_accuracy: 0.1200 - val_loss: 7.2825
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 325s 5s/step - accuracy: 0.1200 - loss: 6.9305 - val_accuracy: 0.1200 - val_loss: 5.8463
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 328s 5s/step - accuracy: 0.1200 - loss: 5.5755 - val_accuracy: 0.1200 - val_loss: 4.8335
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 315s 5s/step - accuracy: 0.1200 - loss: 4.6844 - val_accuracy: 0.1200 - val_loss: 4.2996
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 320s 5s/step - accuracy: 0.1200 - loss: 4.2150 - val_accuracy: 0.1200 - val_loss: 4.0079
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 314s 5s/step - accuracy: 0.1259 - loss: 3.9445 - val_accuracy: 0.1650 - val_loss: 3.7689
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 331s 5s/step - accuracy: 0.1938 - loss: 3.7041 - val_accuracy: 0.2381 - v

✅ Step 1: Save the Full Model


In [41]:
# Save full model (structure + weights + optimizer state)
transformer.save('/content/drive/MyDrive/my_audio2xml_model.h5')
print("✅ Model saved successfully!")


✅ Model saved successfully!


✅ Step 2: Reload the Model for Inference


In [43]:
# from tensorflow.keras.models import load_model

# # Load the model
# model = load_model('/content/drive/MyDrive/my_audio2xml_model.h5')
# print("✅ Model loaded. Ready for inference.")


✅ Step 3: Use Model for Testing on Audio


In [45]:
import librosa
import numpy as np

def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=64)
    mfcc = mfcc.T  # Transpose to shape (time, features)

    # Pad or trim to fixed length (2000, 64)
    if mfcc.shape[0] > 2000:
        mfcc = mfcc[:2000, :]
    else:
        pad_width = 2000 - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')

    return mfcc[np.newaxis, ...]  # Add batch dimension

# Example usage
audio_path = '/content/drive/MyDrive/dataset2/audio/AR_A_fret_0-20.wav'
input_features = preprocess_audio(audio_path)

# Predict
output = transformer.predict(input_features)


ValueError: Layer "functional_18" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(1, 2000, 64) dtype=float32>]

In [46]:
from tensorflow.keras.models import load_model
import numpy as np
import librosa

# ---------------------------
# Step 1: Load the trained model
# ---------------------------
# model = load_model('/content/drive/MyDrive/my_audio2xml_model.h5')
# print("✅ Model loaded.")

# ---------------------------
# Step 2: Define audio preprocessing function
# ---------------------------
def preprocess_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=64)
    mfcc = mfcc.T

    # Pad or trim to (2000, 64)
    if mfcc.shape[0] > 2000:
        mfcc = mfcc[:2000, :]
    else:
        pad_width = 2000 - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')

    return mfcc[np.newaxis, ...]

# ---------------------------
# Step 3: Load input audio
# ---------------------------
audio_path = '/content/drive/MyDrive/dataset2/audio/AR_A_fret_0-20.wav'

# audio_path = '/content/drive/MyDrive/test_audio.wav'
encoder_input = preprocess_audio(audio_path)

# ---------------------------
# Step 4: Inference loop (greedy decoding)
# ---------------------------

# Start decoder input (with <start> token — usually 1 or 0 depending on your tokenizer)
decoder_input = np.array([[start_token]])  # e.g., np.array([[1]])

max_output_length = 50  # set to your max decoder steps
decoded_tokens = []

for i in range(max_output_length):
    predictions = model.predict([encoder_input, decoder_input], verbose=0)
    next_token = np.argmax(predictions[:, -1:, :], axis=-1)

    # Append the predicted token
    decoded_tokens.append(next_token[0, 0])

    # Stop if <end> token is reached
    if next_token[0, 0] == end_token:
        break

    # Append next_token to decoder input for next iteration
    decoder_input = np.concatenate([decoder_input, next_token], axis=1)

# ---------------------------
# Step 5: Convert tokens to text (XML/tab)
# ---------------------------
# Assuming you have a tokenizer or vocabulary reverse mapping:
output_text = ''.join([index_to_token[i] for i in decoded_tokens])
print("Generated Output:\n", output_text)


NameError: name 'start_token' is not defined